# Introduction

<center><img src="https://i.imgur.com/9hLRsjZ.jpg" height=400></center>

This dataset was scraped from [nextspaceflight.com](https://nextspaceflight.com/launches/past/?page=1) and includes all the space missions since the beginning of Space Race between the USA and the Soviet Union in 1957!

### Install Package with Country Codes

In [90]:
# %pip install iso3166

### Upgrade Plotly

Run the cell below if you are working with Google Colab.

In [91]:
# %pip install --upgrade plotly

### Import Statements

In [92]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# These might be helpful:
from iso3166 import countries
from datetime import datetime, timedelta

### Notebook Presentation

In [93]:
pd.options.display.float_format = '{:,.2f}'.format

### Load the Data

In [94]:
df_data = pd.read_csv('mission_launches.csv')

# Preliminary Data Exploration

* What is the shape of `df_data`? 
* How many rows and columns does it have?
* What are the column names?
* Are there any NaN values or duplicates?

In [95]:
print(df_data.shape)
print(df_data.columns)
print(df_data.isna().values.any())
df_data.dropna(inplace=True)

(4324, 9)
Index(['Unnamed: 0.1', 'Unnamed: 0', 'Organisation', 'Location', 'Date',
       'Detail', 'Rocket_Status', 'Price', 'Mission_Status'],
      dtype='object')
True


In [96]:
print(df_data.isna().values.any())

False


In [97]:
df_data.head()

,Unnamed: 0.1,Unnamed: 0,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success
5,5,5,CASC,"LC-9, Taiyuan Satellite Launch Center, China","Sat Jul 25, 2020 03:13 UTC","Long March 4B | Ziyuan-3 03, Apocalypse-10 & N...",StatusActive,64.68,Success


## Data Cleaning - Check for Missing Values and Duplicates

Consider removing columns containing junk data. 

In [98]:
df_data.duplicated().values.any()

False

In [99]:
df_data.drop('Unnamed: 0.1', axis=1, inplace=True)

In [100]:
df_data.drop('Unnamed: 0', axis=1, inplace=True)

In [101]:
df_data.head()

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success
5,CASC,"LC-9, Taiyuan Satellite Launch Center, China","Sat Jul 25, 2020 03:13 UTC","Long March 4B | Ziyuan-3 03, Apocalypse-10 & N...",StatusActive,64.68,Success


## Descriptive Statistics

In [102]:
df_data['Price'] = df_data.Price.astype(str).str.replace(',', '')
df_data['Price'] = pd.to_numeric(df_data.Price)

In [103]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 964 entries, 0 to 4020
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Organisation    964 non-null    object 
 1   Location        964 non-null    object 
 2   Date            964 non-null    object 
 3   Detail          964 non-null    object 
 4   Rocket_Status   964 non-null    object 
 5   Price           964 non-null    float64
 6   Mission_Status  964 non-null    object 
dtypes: float64(1), object(6)
memory usage: 60.2+ KB


In [104]:
df_data.describe()

,Price
count,964.00
mean,153.79
std,288.45
min,5.30
25%,40.00
50%,62.00
75%,164.00
max,"5,000.00"


# Number of Launches per Company

Create a chart that shows the number of space mission launches by organisation.

In [105]:
organisation_df = df_data.groupby('Organisation').agg({'Organisation': pd.Series.count})

In [106]:
launches = px.bar(x=organisation_df.index, 
                  y=organisation_df.Organisation, 
                  labels=({'x': 'Organisation', 'y': 'No of Missions'}), 
                  color=organisation_df.index,
                  color_discrete_sequence=px.colors.qualitative.Set3,
                  log_y=True
                 )
launches.show()

# Number of Active versus Retired Rockets

How many rockets are active compared to those that are decomissioned? 

In [107]:
status = df_data.groupby('Rocket_Status').agg({'Rocket_Status': pd.Series.value_counts})
status

,Rocket_Status
Rocket_Status,
StatusActive,586
StatusRetired,378


In [108]:
status_pie = px.pie(values=status.Rocket_Status, 
                    names=status.index, 
                    title='Rocket Status', 
                    color=status.index, 
                    color_discrete_sequence=px.colors.qualitative.Dark2)
status_pie.show()

# Distribution of Mission Status

How many missions were successful?
How many missions failed?

In [109]:
mission_status = df_data.groupby('Mission_Status').agg({'Mission_Status': pd.Series.value_counts})
mission_status.rename(columns={'Mission_Status': 'No_of_values'}, inplace=True)
mission_status

,No_of_values
Mission_Status,
Failure,36
Partial Failure,17
Prelaunch Failure,1
Success,910


In [110]:
mission_status_pie = px.pie(values=mission_status.No_of_values, 
                            names=mission_status.index, 
                            hole=0.5, 
                            color=mission_status.index, 
                            color_discrete_sequence=px.colors.qualitative.Dark2)
mission_status_pie.show()

# How Expensive are the Launches? 

Create a histogram and visualise the distribution. The price column is given in USD millions (careful of missing values). 

In [111]:
histogram = px.histogram(x=df_data.Price, marginal='histogram')
histogram.show()

In [112]:
df_data.Price.min()

5.3

# Use a Choropleth Map to Show the Number of Launches by Country

* Create a choropleth map using [the plotly documentation](https://plotly.com/python/choropleth-maps/)
* Experiment with [plotly's available colours](https://plotly.com/python/builtin-colorscales/). I quite like the sequential colour `matter` on this map. 
* You'll need to extract a `country` feature as well as change the country names that no longer exist.

Wrangle the Country Names

You'll need to use a 3 letter country code for each country. You might have to change some country names.

* Russia is the Russian Federation
* New Mexico should be USA
* Yellow Sea refers to China
* Shahrud Missile Test Site should be Iran
* Pacific Missile Range Facility should be USA
* Barents Sea should be Russian Federation
* Gran Canaria should be USA


You can use the iso3166 package to convert the country names to Alpha3 format.

In [113]:
country_code = []
for loc in df_data.Location:
    x = loc.split(',')
    x = x[len(x) - 1].strip()
    if x == 'Russia':
        x = 'Russian Federation'
    if x == 'Yellow Sea':
        x = 'China'
    if x == 'Gran Canaria':
        x = 'USA'
    if x == 'Pacific Missile Range Facility':
        x =  'USA'
    if x == 'Shurud Test Missile':
        x = 'Iran'
    code = countries.get(x)[2]
    country_code.append(code)
df_data['Country_Code'] = country_code

In [114]:
country_map = px.choropleth(locations=df_data.Country_Code.value_counts().index, 
                            color=df_data.Country_Code.value_counts(),
                            color_continuous_scale="Viridis",
                            title='No of Launches'
                           )
country_map.show()

# Use a Choropleth Map to Show the Number of Failures by Country


In [115]:
groupby_data = df_data.groupby(['Country_Code', 'Mission_Status'], as_index=False).Mission_Status.value_counts()
country_failure = groupby_data[groupby_data["Mission_Status"] == "Failure"]
country_failure

,Country_Code,Mission_Status,count
0,CHN,Failure,4
3,FRA,Failure,2
6,IND,Failure,5
10,KAZ,Failure,2
13,NZL,Failure,2
15,RUS,Failure,1
18,USA,Failure,20


In [116]:
failure_map = px.choropleth(country_failure, 
                            locations='Country_Code', 
                            color='count', 
                            title='No of Failed Mission By a country',
                            color_continuous_scale='agsunset'
                           )
failure_map.show()

# Create a Plotly Sunburst Chart of the countries, organisations, and mission status. 

In [117]:
sunburst_data = df_data.groupby(['Organisation', 'Country_Code'], as_index=False).Mission_Status.value_counts()

In [118]:
sunburst_chart = px.sunburst(sunburst_data, 
                             path=['Country_Code', 'Organisation', 'Mission_Status'], 
                             values='count', 
                             color_discrete_sequence=px.colors.qualitative.Pastel,
                             title='No of Successful and Failed Space Missions by a organisation in a country'
                            )
sunburst_chart.show()

# Analyse the Total Amount of Money Spent by Organisation on Space Missions

In [119]:
money_spent = df_data.groupby('Organisation', as_index=False).agg({'Price': pd.Series.sum})
money_spent

,Organisation,Price
0,Arianespace,"16,345.00"
1,Boeing,"1,241.00"
2,CASC,"6,340.26"
3,EER,20.00
4,ESA,37.00
5,Eurockot,543.40
6,ExPace,28.30
7,ILS,"1,320.00"
8,ISRO,"2,177.00"
9,JAXA,168.00


In [120]:
money_spent_chart = px.bar(money_spent, 
                           x='Organisation', 
                           y='Price', 
                           log_y=True,
                           labels={'Price': 'Money spent'},
                           color_discrete_sequence=px.colors.qualitative.Prism,
                           color='Organisation'
                          )
money_spent_chart.show()

# Analyse the Amount of Money Spent by Organisation per Launch

In [121]:
average_money_spent = df_data.groupby('Organisation', as_index=False).agg({'Price': pd.Series.mean})

In [122]:
average_money_spent_cht = px.bar(average_money_spent, 
                                 x='Organisation',
                                 y='Price',
                                 color='Organisation',
                                 labels={'Price': 'Average Money Spent per launch'},
                                 log_y=True,
                                 color_discrete_sequence=px.colors.qualitative.Safe
                                )
average_money_spent_cht.show()

In [142]:
df_data['Date'] = pd.to_datetime(df_data['Date'], utc=True).dt.tz_localize(None)

# Chart the Number of Launches per Year

In [155]:
launches_year = df_data.groupby(df_data['Date'].dt.year).agg({'Date': pd.Series.count})

In [159]:
launches_cht = px.bar(x=launches_year.index, 
                      y=launches_year.Date, 
                      color=launches_year.index, 
                      labels={'x': 'Year', 'y': 'No of Launches'})
launches_cht.show()

# Chart the Number of Launches Month-on-Month until the Present

Which month has seen the highest number of launches in all time? Superimpose a rolling average on the month on month time series chart. 

In [165]:
month_data = df_data['Date'].dt.strftime('%b').value_counts()

In [168]:
month_chart = px.bar(x=month_data.index,
                     y=month_data.values,
                     color=month_data.index,
                     labels={'x': 'Month', 'y': 'No of launches'},
                     color_discrete_sequence=px.colors.qualitative.T10
                    )
month_chart.show()

# Launches per Month: Which months are most popular and least popular for launches?

Some months have better weather than others. Which time of year seems to be best for space missions?

# How has the Launch Price varied Over Time? 

Create a line chart that shows the average price of rocket launches over time. 

# Chart the Number of Launches over Time by the Top 10 Organisations. 

How has the dominance of launches changed over time between the different players? 

# Cold War Space Race: USA vs USSR

The cold war lasted from the start of the dataset up until 1991. 

## Create a Plotly Pie Chart comparing the total number of launches of the USSR and the USA

Hint: Remember to include former Soviet Republics like Kazakhstan when analysing the total number of launches. 

## Create a Chart that Shows the Total Number of Launches Year-On-Year by the Two Superpowers

## Chart the Total Number of Mission Failures Year on Year.

## Chart the Percentage of Failures over Time

Did failures go up or down over time? Did the countries get better at minimising risk and improving their chances of success over time? 

# For Every Year Show which Country was in the Lead in terms of Total Number of Launches up to and including including 2020)

Do the results change if we only look at the number of successful launches? 

# Create a Year-on-Year Chart Showing the Organisation Doing the Most Number of Launches

Which organisation was dominant in the 1970s and 1980s? Which organisation was dominant in 2018, 2019 and 2020? 